In [ ]:
# Importar librerías
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# Cargar datos
col_names = ['unit', 'time', 'op1', 'op2', 'op3'] + [f'sensor_{i}' for i in range(1, 22)] # Nombre de las columnas usadas en la variable train, usado para leer el archivos

train = pd.read_csv(
    "../data/CMAPSSData/train_FD001.txt", # Ruta del archivo
    sep=r'\s+',
    header=None, # Sin header
    names=col_names # Nombres de la lista anterior
) # Indacamos de la ruta de donde tenemos los datos


# Crear RUL (Remaining Useful Life)
train['RUL'] = train.groupby('unit')['time'].transform(max) - train['time'] # Creamos el rul, agrupandolo por unit, calculando el ciclo(time) maximo - el ciclo(time) actual


# Feature Engineering
def create_features(df, window_sizes=[5, 10]):
    df_feat = df.copy() # Creamos una copia de DF que es el train
    
    # Diferencias de sensores
    sensor_cols = [c for c in df.columns if 'sensor_' in c] # Recogemos los sensores de los motores
    for col in sensor_cols:
        df_feat[f'{col}_diff'] = df_feat.groupby('unit')[col].diff() # Agrupamos los motores y buscamos la diferencia de ciclo
    
    # Medias y desviaciones en ventanas móviles
    for window in window_sizes: # Bucle usando el windows que le pasamos por los metodos
        for col in sensor_cols:
            df_feat[f'{col}_mean_{window}'] = df_feat.groupby('unit')[col].rolling(window=window, min_periods=1).mean().reset_index(0,drop=True) # Calcula la media de los motores
            df_feat[f'{col}_std_{window}']  = df_feat.groupby('unit')[col].rolling(window=window, min_periods=1).std().reset_index(0,drop=True) # Calcula la desviacion de los motores 
    
    return df_feat # Devolvemos los datos 

train_feat = create_features(train) # Guardamos los datos en una variable

# Escalado / Normalización
feature_cols = [c for c in train_feat.columns if 'sensor' in c or 'op' in c] # Creamos una lista con las columnas que tenga sensor o op 
scaler = StandardScaler() # Creamos una scaler haciendo uso de sklearn
train_feat[feature_cols] = scaler.fit_transform(train_feat[feature_cols]) # Calculamos la media y la desviacion y le aplicamos la formula de normalizacion

# Guardar dataset procesado
train_feat.to_csv("../data/CMAPSSData/train_FD001_features.csv", index=False)

# Vista previa
train_feat.head()